In [1]:
import pandas as pd
import json
import requests
import os
import re

In [2]:
def download_dicionario_variaveis(fname):
    
    url = ('http://dados.prefeitura.sp.gov.br/dataset/'
        'bf5df0f4-4fb0-4a5e-b013-07d098cc7b1c/'
        'resource/98520909-4e3a-4043-8188-01523126f7d8/'
        'download/notas-explicativas-10.txt')

    with requests.get(url) as r:
        dados = r.text
    
    with open(fname, 'w', encoding='utf-8') as f:
        f.write(dados)
    
def pegar_dicionario_variaveis():
    
    
    fname = 'dicionario_variaveis_cadastro_servidores_ativos.txt'
    if os.path.exists(fname):
        with open(fname, encoding='utf-8') as f:
            return f.read()
    else:
        download_dicionario_variaveis(fname)
        return pegar_dicionario_variaveis()

In [3]:
dici = pegar_dicionario_variaveis()

In [4]:
print(dici)

NOTAS EXPLICATIVAS - Atualizadas em 05/07/2021

1) Constam servidores(as) ativos(as) da Administra��o Direta na data de gera��o da base: admitidos(as), contratados(as) por tempo determinado, efetivos(as), em comiss�o e requisitados(as).

2) Explica��es sobre os campos da base:

	CAMPO			DESCRI��O

	NOME			Nome do(a) servidor(a)
	CARGO_BASICO		Cargo/fun��o base do(a) servidor(a)
	REF_CARGO_BAS		Padr�o, Refer�ncia ou S�mbolo do cargo/fun��o base do(a) servidor(a)
	SEGMENTO		Segmentos dos Quadros de Pessoal dos N�veis B�sico e M�dio; Disciplinas/Forma��es dos Quadros de Pessoal de N�vel Superior, de Analistas da Administra��o P�blica Municipal e de Profissionais de Engenharia, Arquitetura, Agronomia e Geologia; Atividades do Quadro dos Profissionais de Gest�o Governamental; Classes do Quadro de Pessoal da Educa��o; Naturezas do Quadro do Pessoal da Administra��o Tribut�ria; Especialidades/Disciplinas/Atividades/Segmentos do Quadro da Sa�de
	GRUPO			Quadro funcional a qual o cargo do(a) se

In [5]:
serv_ativos = pd.read_csv('original_data/aggregated/cadastro_servidores_ativos_jul_2021.csv',
                         sep=';', encoding='utf-8')

In [6]:
serv_ativos.head()

,REGISTRO,VINCULO,NOME,CARGO_BASICO,REF_CARGO_BAS,SEGMENTO,GRUPO,SUBGRUPO,ESCOL_CARGO_BASICO,CARGO_COMISSAO,...,ESCOL_CARGO_COMISSAO,DATA_INICIO_EXERC,REL_JUR_ADM,SECRET_SUBPREF,SETOR,ORGAO_EXT,SEXO,ANO_NASCIMENTO,RACA,DEFICIENTE
0,1145541,15,CLEUZA BORGES PEREIRA SILVA,ASSESSOR TECNICO II,DAS12,NaN,QPA,CARGO EM COMISSAO,SUPERIOR COMPLETO,NaN,...,NaN,15/09/2010,EM COMISSAO,SECRETARIA DE GOVERNO MUNICIPAL,ASSESSORIA TECNICA DE ASSUNTOS JURIDICOS - PES...,NaN,F,1949,PARDA,NAO
1,1146254,2,ROSELI LUZIA COPULA,PROFESSOR ED INFANTIL E ENS FUNDAMENTAL CAT 3,QPE23E,NaN,QPE LEI 14660/07,DOCENTE,LICENCIATURA PLENA COMPLETA,NaN,...,NaN,21/09/1995,EFETIVO,SECRETARIA MUNICIPAL DE EDUCACAO,EMEF DESEMBARGADOR FRANCISCO MEIRELLES,NaN,F,1949,BRANCA,NAO
2,1150669,2,LENIR BILHORA DA ROCHA,PROFESSOR ED INFANTIL E ENS FUNDAMENTAL CAT 3,QPE21E,NaN,QPE LEI 14660/07,DOCENTE,LICENCIATURA PLENA COMPLETA,NaN,...,NaN,30/09/1999,EFETIVO,SECRETARIA MUNICIPAL DE EDUCACAO,EMEF EDGARD CAVALHEIRO,NaN,F,1947,BRANCA,NAO
3,1150995,2,CIBELE DE NAPOLES GALATI,PROFESSOR ED INFANTIL E ENS FUNDAMENTAL CAT 3,QPE21E,NaN,QPE LEI 14660/07,DOCENTE,LICENCIATURA PLENA COMPLETA,NaN,...,NaN,12/03/1997,EFETIVO,SECRETARIA MUNICIPAL DE EDUCACAO,EMEI REGENTE FEIJO,NaN,F,1950,BRANCA,NAO
4,1153412,4,ANA MARIA CALIXTO,SUPERVISOR ESCOLAR,QPE22E,NaN,QPE LEI 14660/07,GESTOR,LICENCIATURA PLENA COMPLETA,NaN,...,NaN,30/04/1996,EFETIVO,SECRETARIA MUNICIPAL DE EDUCACAO,SUPERVISAO ESCOLAR,NaN,F,1948,BRANCA,NAO


In [7]:
serv_ativos['SUBGRUPO'].unique()

array(['CARGO EM COMISSAO', 'DOCENTE', 'GESTOR', 'SUPERIOR', 'MEDIO',
       'ATIVID ARTISTICA', 'BASICO', 'APOIO MEDIO', 'APOIO BASICO', 'QTG',
       'APOIO', 'MEDIO TECNICO', 'PROCURADOR', 'QPG', 'CLT - AHM',
       'REQUISITADO', 'OPERACIONAL', 'ASSISTENCIA MILITAR', 'QGC',
       'PREFEITO'], dtype=object)

In [8]:
superior = serv_ativos['SUBGRUPO']=='SUPERIOR'

In [9]:
serv_ativos[superior]['GRUPO'].unique()

array(['QAV', 'QAA', 'QEAG', 'QS', 'PCCS', 'QPDU', 'QPA', 'QPCEL', 'QPAT',
       'QPS', 'QPS LEI 14713/08', 'QPF', 'QPGG'], dtype=object)

In [10]:
não_seleiconados = {
    'QAV' : 'agentes vistores',
    'QS' : 'saúde - inclui médicos',
    'QPS' : 'saúde - inclui médicos',
    'QPS LEI 14713/08' : 'saúde - inclui médicos',
    'QPF' : 'fiscalização'
}

In [11]:
grupos_selecionados = {
    'QAA' : 'Analistas da Administração Pública Municipal',
    'QEAG' : 'Profissionais de Engenharia, Arquitetura, Agronomia e Geologia',
    'PCCS' : 'Pessoal de Nível Básico, Médio e Superior da Prefeitura do Município de São Paulo',
    'QPDU' : 'Profissionais do Desenvolvimento Urbano',
    'QPA' : 'Profissionais da Administração',
    'QPCEL' : 'Profissionais da Cultura, Esporte e Lazer',
    'QPAT' : 'Pessoal da Administração Tributária do Município de São Paulo',
    'QPGG' : 'Profissionais de Gestão Governamental',
    }

In [12]:
filtro_grupos = serv_ativos['GRUPO'].isin(grupos_selecionados)

In [13]:
serv_ativos[superior&filtro_grupos]['ESCOL_CARGO_BASICO'].unique()

array(['SUPERIOR COMPLETO'], dtype=object)

In [14]:
quadros_tecnicos = serv_ativos[superior&filtro_grupos].copy().reset_index(drop=True)

In [15]:
for cargo in quadros_tecnicos['CARGO_BASICO'].unique():
    print(cargo)

ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL IV
PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA NIVEL IV
ANALISTA DE ORDENAMENTO TERRITORIAL NIVEL IV
PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA NIVEL I
PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA NIVEL II
ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL II
ESP DESENVOLVIMENTO URBANO NÍVEL III
ANALISTA DE INFORMACOES CULTURA E DESPORTO
ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL I
SOCIOLOGO CLASSE I
ANALISTA
ESP DESENVOLVIMENTO URBANO NÍVEL II
ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL NIVEL IV
ANALISTA PLANEJAMENTO DESENV ORGANIZACIONAL NIVEL IV
ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL NIVEL III
PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA NIVEL III
ANALISTA DE MEIO AMBIENTE NIVEL II
ANALISTA PLANEJAMENTO DESENV ORGANIZACIONAL NIVEL III
ECONOMISTA CLASSE I
ANALISTA DE INFORMACOES CULTURA E DESPORTO NIVEL III
HISTORIADOR CLASSE I
ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL NIVEL I
ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL NIVEL II
ANAL

In [16]:
def replace_nivel(cargo):
    
    patt = '(N[IÍ]VEL)|(CLASSE)'
    busca = re.search(patt, cargo)
    if busca:
        achado = busca.group()
        cargo = cargo.replace(achado, '*')
    return cargo

In [17]:
def repescagem_replace_nivel(cargo):
    
    split = cargo.split(' ')
    if len(split)>1:
        inicio = ' '.join(split[:-1])
        final = split[-1]
        final = final.strip()
        if final.startswith('N.'):
            final = final.replace('N.', '*')
        elif final.startswith('N'):
            final = final.replace('N', '*')
        
        return f'{inicio} {final}'
    else:
        return cargo

In [18]:
def replace_nivel_all(cargo):
    
    cargo = replace_nivel(cargo)
    cargo = repescagem_replace_nivel(cargo)
    
    return cargo

In [19]:
def pegar_nivel(cargo):
    
    cargo = replace_nivel_all(cargo)
    splited = cargo.split('*')
    if len(splited)>1:
        return splited[-1].strip()
    else:
        return 'Nenhum'

In [20]:
quadros_tecnicos['nivel_limpo']=quadros_tecnicos['CARGO_BASICO'].apply(pegar_nivel)

In [21]:
quadros_tecnicos['nivel_limpo'].unique()

array(['IV', 'I', 'II', 'III', 'Nenhum'], dtype=object)

In [22]:
check_nivel = pd.crosstab(quadros_tecnicos['CARGO_BASICO'], quadros_tecnicos['nivel_limpo'])

In [23]:
check_nivel.head()

nivel_limpo,I,II,III,IV,Nenhum
CARGO_BASICO,,,,,
ANALISTA,0,0,0,0,33
ANALISTA ASSIST DESENV SOCIAL - EQUIP SOCIAL NIVEL I,2,0,0,0,0
ANALISTA ASSIST DESENV SOCIAL - EQUIP SOCIAL NIVEL II,0,10,0,0,0
ANALISTA ASSIST DESENV SOCIAL - EQUIP SOCIAL NIVEL III,0,0,2,0,0
ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL,0,0,0,0,2


In [24]:
for col in check_nivel.columns:
    check_nivel[col] = check_nivel[col].apply(lambda x: True if x else False)

In [25]:
(check_nivel.apply(lambda row: row.sum(), axis =1)<2).all()

True

In [26]:
def limpar_cargo(cargo):
    
    cargo = replace_nivel_all(cargo)
    
    cargo = cargo.split('*')[0]
    cargo = cargo.replace('-', '')
    cargo = cargo.replace('  ', ' ')
    
    return cargo.strip()

In [27]:
quadros_tecnicos['cargo_limpo'] = quadros_tecnicos['CARGO_BASICO'].apply(limpar_cargo)

In [28]:
quadros_tecnicos['cargo_limpo'].unique()

array(['ANALISTA DE INFORMACOES CULTURA E DESPORTO',
       'PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA',
       'ANALISTA DE ORDENAMENTO TERRITORIAL',
       'ESP DESENVOLVIMENTO URBANO', 'SOCIOLOGO', 'ANALISTA',
       'ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL',
       'ANALISTA PLANEJAMENTO DESENV ORGANIZACIONAL',
       'ANALISTA DE MEIO AMBIENTE', 'ECONOMISTA', 'HISTORIADOR',
       'DIRETOR DE CRECHE', 'ESPECIALISTA',
       'AUDITOR FISCAL TRIBUTARIO MUNICIPAL', 'TECNOLOGO EM ELETRICIDADE',
       'TECNOLOGO EM CONSTRUCAO CIVIL', 'CONTADOR',
       'ESP ADM ORÇAMENTO E FINANÇAS PÚBLICAS', 'ENGENHEIRO', 'ARQUITETO',
       'GEOLOGO', 'ESP ASSISTÊNCIA E DESENVOLVIMENTO SOCIAL',
       'ESP INFORM TECN CULTURAIS E DESPORTIVAS',
       'ANALISTA ASSIST DESENV SOCIAL EQUIP SOCIAL',
       'ANALISTA POLITICAS PUBLICAS GESTAO GOVERNAMENTAL',
       'AUDITOR MUNICIPAL DE CONTROLE INTERNO'], dtype=object)

In [29]:
remover_cargos = [
    'TECNOLOGO EM ELETRICIDADE',
    'TECNOLOGO EM CONSTRUCAO CIVIL',
    'DIRETOR DE CRECHE'
]

In [30]:
quadros_tecnicos = quadros_tecnicos[~quadros_tecnicos['cargo_limpo'].isin(remover_cargos)].reset_index(drop=True).copy()

In [31]:
for cargo in quadros_tecnicos['cargo_limpo'].unique():
    print(cargo)

ANALISTA DE INFORMACOES CULTURA E DESPORTO
PROFISSIONAL ENG, ARQ, AGRONOMIA,GEOLOGIA
ANALISTA DE ORDENAMENTO TERRITORIAL
ESP DESENVOLVIMENTO URBANO
SOCIOLOGO
ANALISTA
ANALISTA ASSISTENCIA DESENVOLVIMENTO SOCIAL
ANALISTA PLANEJAMENTO DESENV ORGANIZACIONAL
ANALISTA DE MEIO AMBIENTE
ECONOMISTA
HISTORIADOR
ESPECIALISTA
AUDITOR FISCAL TRIBUTARIO MUNICIPAL
CONTADOR
ESP ADM ORÇAMENTO E FINANÇAS PÚBLICAS
ENGENHEIRO
ARQUITETO
GEOLOGO
ESP ASSISTÊNCIA E DESENVOLVIMENTO SOCIAL
ESP INFORM TECN CULTURAIS E DESPORTIVAS
ANALISTA ASSIST DESENV SOCIAL EQUIP SOCIAL
ANALISTA POLITICAS PUBLICAS GESTAO GOVERNAMENTAL
AUDITOR MUNICIPAL DE CONTROLE INTERNO


In [32]:
quadros_tecnicos.to_csv('quadros_tecnicos_pmsp.csv',
                        sep=';',
                        index=False,
                        encoding='utf-8')